In [9]:
import pyarrow
import pyarrow.parquet as pq
import pandas as pd
from datetime import datetime as dt
import os
import dask.dataframe as dd


In [ ]:
# Technique used was to read in .csv file, create a column of date in datetime format
# then write the dataframe to a parquet file. This is done for each .csv file in the directory
# could not do df.to_parquet() with partioning_cols by Date because of limit on number of partitions because some files had more than 1024 dates, which is limit on partitions
# so had to write to parquet file using df.grouby date so each file is one day for a given ticker
# could not do on google colab even with high ram because the writing to google drive would be too slow, would have taken 1 week to run
# so ran on local machine (took 17 hours for 7600 tickers) and uploaded to google drive

In [11]:
# Define the folder where the .txt files are located
#folder_path_source = "G:/My Drive/Backtesting/FirstRateData/Stocks/stocks1min-txt"
folder_path_source = "C:/FirstRateDataTest/stocks1min-csv"
#folder_path_destination = "G:/My Drive/Backtesting/FirstRateData/Stocks/stocks1min-csv/"
csv_folder_path_destination = "C:/FirstRateDataTest/modified_csv/"
#parquet_folder_path_destination = "C:/FirstRateDataTest/parquet/"


#os.makedirs(base_dir, exist_ok=True)

In [12]:
start = dt.now()
# Loop through all files in the folder
for filename in os.listdir(folder_path_source):
    # Check if the file is a .txt file
    if filename.endswith('.csv'):
        csv_file_path = os.path.join(folder_path_source, filename)
        filename_str = str(filename)
        ticker = filename_str.split('_')[0]
        #print('ticker:', ticker)
        #parquet_file_path = parquet_folder_path_destination + os.path.splitext(filename)[0] + '.parquet'
        new_csv_file_path = csv_folder_path_destination + os.path.splitext(filename)[0] + '.csv'
        try:
            # Read the .txt file into a DataFrame
            # Adjust the delimiter based on your file format, e.g., ',' or '\t'
            df = pd.read_csv(csv_file_path)
            df['Datetime'] = pd.to_datetime(df['Datetime'])
            df['Date'] = df['Datetime'].dt.date

            df['Date'] = pd.to_datetime(df['Date'])
            
            
            # reorder columns so that Date is first
            df = df[['Date', 'Datetime', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker']]
            if 'Unnamed: 0' in df.columns:
                df = df.drop(columns=['Unnamed: 0'])
            
            # Save the DataFrame to a .paraquet file, save first to csv file 
            #df.to_parquet(parquet_file_path, partition_cols='Date', index=False)
            #df.to_csv(new_csv_file_path, index=False)
           

            #print(f"Converted {filename} ")

            # Create output directory
            base_dir = f"C:/FirstRateDataTest/stocks1min-parquet/{ticker}_1min_parquet/"
            
            os.makedirs(base_dir, exist_ok=True)

            # Group by partition columns
            for thedate, group in df.groupby(["Date"]):
                # Create subdirectory
                date_str = list(thedate)[0].strftime("%Y-%m-%d")
                #sub_dir = os.path.join(base_dir, f"{date_str}_{ticker}_1min")
                
                
                # Write custom filename
                group.to_parquet(os.path.join(base_dir, f"{date_str}_{ticker}_1min.parquet"))
            #df.to_parquet(base_dir, partition_cols='Date')


        except Exception as e:
            print(f"Failed to convert {filename}: {e}")

print("Conversion complete!")
end = dt.now()
print('start:', start)
print('end:', end)



Conversion complete!
start: 2024-12-30 15:16:00.263836
end: 2024-12-31 08:16:22.628413


In [6]:
dd_AMD = dd.read_parquet('C:/FirstRateDataTest/AMD_1min_parquet')

In [7]:
dd_AMD.head()

,Date,Datetime,Open,High,Low,Close,Volume,Ticker
0,2000-01-03,2000-01-03 09:39:00,29.9375,29.9375,29.6250,29.9375,12300,AMD
1,2000-01-03,2000-01-03 09:40:00,29.6875,29.9375,29.6875,29.8750,14400,AMD
2,2000-01-03,2000-01-03 09:41:00,29.6875,29.8750,29.6875,29.8750,700,AMD
3,2000-01-03,2000-01-03 09:42:00,29.7500,29.8750,29.7500,29.7500,10500,AMD
4,2000-01-03,2000-01-03 09:43:00,29.7500,29.8750,29.7500,29.8750,9800,AMD


In [8]:
dd_AMD.npartitions

6275

In [36]:
# Group by partition columns
for thedate, group in df.groupby(["Date"]):
    # Create subdirectory
    date_str = list(thedate)[0].strftime("%Y-%m-%d")
    sub_dir = os.path.join(base_dir, f"{date_str}_{ticker}")
    os.makedirs(sub_dir, exist_ok=True)
    
    # Write custom filename
    group.to_parquet(os.path.join(sub_dir, f"{date_str}_{ticker}.parquet"))

In [4]:
df.head(5)

,Date,Datetime,Open,High,Low,Close,Volume,Ticker
0,2000-01-03,2000-01-03 09:31:00,105.0625,105.3750,104.8750,104.9375,13000,AAPL
1,2000-01-03,2000-01-03 09:32:00,104.8750,105.3125,104.8750,105.1250,35900,AAPL
2,2000-01-03,2000-01-03 09:33:00,105.0000,105.3125,104.6250,105.0000,36300,AAPL
3,2000-01-03,2000-01-03 09:34:00,105.0625,105.1250,104.5625,104.5625,79200,AAPL
4,2000-01-03,2000-01-03 09:35:00,105.0000,105.1250,104.3750,104.4375,21600,AAPL


In [5]:
df.tail(5)

,Date,Datetime,Open,High,Low,Close,Volume,Ticker
3814189,2024-12-10,2024-12-10 19:50:00,247.70,247.70,247.70,247.70,200,AAPL
3814190,2024-12-10,2024-12-10 19:51:00,247.68,247.68,247.68,247.68,340,AAPL
3814191,2024-12-10,2024-12-10 19:53:00,247.70,247.70,247.70,247.70,236,AAPL
3814192,2024-12-10,2024-12-10 19:56:00,247.71,247.71,247.71,247.71,100,AAPL
3814193,2024-12-10,2024-12-10 19:59:00,247.70,247.70,247.70,247.70,202,AAPL


In [31]:
trading_date = '2024-12-10'
start = dt.now()
df_csv = pd.read_csv(new_csv_file_path)
#print(df_csv.tail(5))
#print(df_csv[df_csv['Date'].str.contains(trading_date)].head(5))
print(df_csv[df_csv['Date']==trading_date])
end = dt.now()
print('start:', start)
print('end:', end)
print('time:', end-start)

               Date             Datetime    Open    High     Low   Close  \
3813489  2024-12-10  2024-12-10 04:00:00  246.60  246.74  246.53  246.64   
3813490  2024-12-10  2024-12-10 04:01:00  246.68  246.72  246.68  246.70   
3813491  2024-12-10  2024-12-10 04:02:00  246.84  246.84  246.84  246.84   
3813492  2024-12-10  2024-12-10 04:03:00  246.74  246.74  246.72  246.72   
3813493  2024-12-10  2024-12-10 04:04:00  246.82  246.86  246.82  246.86   
...             ...                  ...     ...     ...     ...     ...   
3814189  2024-12-10  2024-12-10 19:50:00  247.70  247.70  247.70  247.70   
3814190  2024-12-10  2024-12-10 19:51:00  247.68  247.68  247.68  247.68   
3814191  2024-12-10  2024-12-10 19:53:00  247.70  247.70  247.70  247.70   
3814192  2024-12-10  2024-12-10 19:56:00  247.71  247.71  247.71  247.71   
3814193  2024-12-10  2024-12-10 19:59:00  247.70  247.70  247.70  247.70   

         Volume Ticker  
3813489    1485   AAPL  
3813490    1098   AAPL  
3813491     

In [32]:
trading_date = '2024-12-10'
start = dt.now()
df_parquet = pq.read_table(parquet_file_path).to_pandas()
print(df_parquet[df_parquet['Date']==trading_date])
end = dt.now()
print('start:', start)
print('end:', end)
print('time:', end-start)

               Date             Datetime    Open    High     Low   Close  \
3813489  2024-12-10  2024-12-10 04:00:00  246.60  246.74  246.53  246.64   
3813490  2024-12-10  2024-12-10 04:01:00  246.68  246.72  246.68  246.70   
3813491  2024-12-10  2024-12-10 04:02:00  246.84  246.84  246.84  246.84   
3813492  2024-12-10  2024-12-10 04:03:00  246.74  246.74  246.72  246.72   
3813493  2024-12-10  2024-12-10 04:04:00  246.82  246.86  246.82  246.86   
...             ...                  ...     ...     ...     ...     ...   
3814189  2024-12-10  2024-12-10 19:50:00  247.70  247.70  247.70  247.70   
3814190  2024-12-10  2024-12-10 19:51:00  247.68  247.68  247.68  247.68   
3814191  2024-12-10  2024-12-10 19:53:00  247.70  247.70  247.70  247.70   
3814192  2024-12-10  2024-12-10 19:56:00  247.71  247.71  247.71  247.71   
3814193  2024-12-10  2024-12-10 19:59:00  247.70  247.70  247.70  247.70   

         Volume Ticker  
3813489    1485   AAPL  
3813490    1098   AAPL  
3813491     